# CARWatch – Saliva Data Processing

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.utils.time import time_to_timedelta
from biopsykit.utils.dataframe_handling import multi_xs, int_from_str_idx, camel_to_snake

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.datasets import CarWatchDatasetRaw

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
deploy_type = "develop"

In [ ]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
base_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = base_path.joinpath("saliva/processed")
export_path.mkdir(exist_ok=True)

In [ ]:
dataset = CarWatchDatasetRaw(base_path)

## Load Data

### Saliva

In [ ]:
cort_long = dataset.cortisol
cort_long = cort_long.join(dataset.condition_map)
cort_long = cort_long.reset_index().set_index(["subject", "night", "condition", "sample"])
cort_long.head()

### Merged Sleep Information

In [ ]:
sleep_info = dataset.sleep_information_merged
sleep_info.head()

## Saliva Processing

### Drop SA sample

In [ ]:
cort_long = cort_long.drop("SA", level="sample", errors="ignore")
cort_long.head()

### Assign Recording Dates to Saliva Samples

In [ ]:
if "date" not in cort_long.columns:
    cort_long = cort_long.join(dataset.date)
cort_long.head()

### Convert Saliva Times into Minutes relative to S0

In [ ]:
cort_long["wake_onset_time"] = pd.to_timedelta(sleep_info["wake_onset_time"])
cort_long["time"] = bp.saliva.utils.sample_times_datetime_to_minute(cort_long["time_abs"])
cort_long["time"] = cort_long["time"].round(3)
cort_long["time_abs"] = time_to_timedelta(cort_long["time_abs"])

cort_long.head()

## Feature Computation

In [ ]:
cort_auc = bp.saliva.auc(cort_long, saliva_type="cortisol", remove_s0=False)
cort_auc.head()

In [ ]:
cort_inc = bp.saliva.max_increase(cort_long, saliva_type="cortisol", remove_s0=False)
cort_inc.head()

In [ ]:
cort_slope = bp.saliva.slope(cort_long, sample_idx=[0, 3], saliva_type="cortisol")
cort_slope = cort_slope.join(bp.saliva.slope(cort_long, sample_idx=[0, 4], saliva_type="cortisol"))
# cort_slope = cort_slope.join(bp.saliva.slope(cort_long, sample_idx=[2, 4], saliva_type="cortisol"))
cort_slope.head()

In [ ]:
cort_max = bp.saliva.max_value(cort_long, saliva_type="cortisol")
cort_max.head()

In [ ]:
cort_ini = bp.saliva.initial_value(cort_long, saliva_type="cortisol")
cort_ini.head()

In [ ]:
cort_feat = pd.concat([cort_auc, cort_inc, cort_slope, cort_max, cort_ini], axis=1)
cort_feat = bp.saliva.utils.saliva_feature_wide_to_long(cort_feat, "cortisol")
cort_feat.head()

## Export

In [ ]:
# Move cortisol column to last position in dataframe

columns = list(cort_long.columns)
cort_out = cort_long[columns[1:] + [columns[0]]]
cort_out.head()

In [ ]:
export_path = base_path.joinpath("saliva/processed")

In [ ]:
cort_out.to_csv(export_path.joinpath("cortisol_samples.csv"))
cort_feat.to_csv(export_path.joinpath("cortisol_features.csv"))